In [1]:
from plotly.offline import iplot,plot, init_notebook_mode

## Visualization 

### Overview

Visualization poses a significant challenge for the space weather community: output from models and data are very domain-specific, both in content (coordinate systems, units) and in representation (file formats and data structures). On the other hand, science users also have their preferred context for analyzing these results - for instance, they may only want simulation results interpolated on a satellite trajectory and in a specific coordinate system with their own prefered units.

Kamodo aims to strike a balance between the intent of the model (or data) provider and the goals of the user, by making it easy for developers to provide context for their output and for users to easily change that context. It accomplishes this in two ways:

* By leveraging default arguments given by model and data providers 
* By mapping the shape of function inputs and output to certain registered plot types

This strategy allows Kamodo to automatically generate plots for arbitrary model output and data sources, while still allowing for customization by the end user.

### Available Plot Types

Kamodo keeps a registry of plotting functions, indexed by argument shape and output shape.

In [2]:
from kamodo.plotting import plot_types
plot_types

plot_type  \
out_shape arg_shapes                                       
(1,)      ((N, M), (N, M), (N, M))         3d-parametric   
(N,)      ((N,),)                                1d-line   
          ((N,), (N,), (N,))              3d-line-scalar   
(N, 2)    ((N,),)                                2d-line   
          ((N, 2),)                            2d-vector   
(N, 3)    ((N,),)                                3d-line   
          ((N, 3),)                            3d-vector   
(N, M)    ((N,), (M,))                        2d-contour   
          ((N, M), (N, M))               2d-contour-skew   
          ((N, M), (N, M), (N, M))  3d-parametric-scalar   
          ((1,), (N, M), (N, M))                3d-plane   
          ((N, M), (1,), (N, M))                3d-plane   
          ((N, M), (N, M), (1,))                3d-plane   
(N, M, 1) ((1,), (N,), (M,))                    3d-plane   
          ((N,), (1,), (M,))                    3d-plane   
          ((N,), (M,), (1,))                    3d-plane   

                                                                  function  
out_shape arg_shapes                                                        
(1,)      ((N, M), (N, M), (N, M))       <function surface at 0x131cf4620>  
(N,)      ((N,),)                      <function line_plot at 0x120b98730>  
          ((N,), (N,), (N,))           <function line_plot at 0x120b98730>  
(N, 2)    ((N,),)                      <function line_plot at 0x120b98730>  
          ((N, 2),)                  <function vector_plot at 0x121fb57b8>  
(N, 3)    ((N,),)                      <function line_plot at 0x120b98730>  
          ((N, 3),)                  <function vector_plot at 0x121fb57b8>  
(N, M)    ((N,), (M,))              <function contour_plot at 0x131cf4488>  
          ((N, M), (N, M))          <function contour_plot at 0x131cf4488>  
          ((N, M), (N, M), (N, M))       <function surface at 0x131cf4620>  
          ((1,), (N, M), (N, M))           <function plane at 0x131cf4598>  
          ((N, M), (1,), (N, M))           <function plane at 0x131cf4598>  
          ((N, M), (N, M), (1,))           <function plane at 0x131cf4598>  
(N, M, 1) ((1,), (N,), (M,))               <function plane at 0x131cf4598>  
          ((N,), (1,), (M,))               <function plane at 0x131cf4598>  
          ((N,), (M,), (1,))               <function plane at 0x131cf4598>

When a user tries to plot a given variable, a lookup is made into the table above and the corresponding plotting function is used to generate the output. Examples below demonstrate the intended workflow.

### 1-Dimensional line plots

In [3]:
from kamodo.kamodo import Kamodo, kamodofy
kamodo = Kamodo('g_N[kg] = x_N**2')
kamodo['f[g]'] = 'g_N'
kamodo

Kamodo([(g_N(x_N), <function _lambdifygenerated(x_N)>),
        (g_N, <function _lambdifygenerated(x_N)>),
        (f(x_N), <function _lambdifygenerated(x_N)>),
        (f, <function _lambdifygenerated(x_N)>)])

Here we have defined a function $g_N$ which returns an array of shape $N$. As input, it takes one argument $x_N$ which also has size $N$. We also generate a function $f$ which is the same as $g_N$ but with a different units.

!!! note
    We could have named the function $g$ instead of $g_N$. The variable names have no bearing on the resulting plots - only the argument input shapes and output shapes matter.

When we call Kamodo's plot function, we define which variable we are plotting and domain over which the arguments are applied:

In [4]:
import numpy as np
import plotly.io as pio

fig = kamodo.plot(f = dict(x_N = np.linspace(-4, 3, 30)))
pio.write_image(fig, 'images/1d-line.svg')

![1d](images/1d-line.svg?1)

This is the graph $f(x_n)$ for $x_N \in [-4,3]$.

### Time series data
The process for time series data is the same, except we use a pandas datetime index for the input argument.

In [5]:
import pandas as pd
t_N = pd.date_range('Nov 9, 2018', 'Nov 20, 2018', freq = 'H')

In [6]:
@kamodofy(units = 'kg/m^3')
def rho_N(t_N = t_N):
    dt_days = (t_N - t_N[0]).total_seconds()/(24*3600)
    return 1+np.sin(dt_days) + .1*np.random.random(len(dt_days))

kamodo = Kamodo(rho_N = rho_N, verbose = False)
kamodo

Kamodo([(rho_N(t_N),
         <function __main__.rho_N(t_N=DatetimeIndex(['2018-11-09 00:00:00', '2018-11-09 01:00:00',
               '2018-11-09 02:00:00', '2018-11-09 03:00:00',
               '2018-11-09 04:00:00', '2018-11-09 05:00:00',
               '2018-11-09 06:00:00', '2018-11-09 07:00:00',
               '2018-11-09 08:00:00', '2018-11-09 09:00:00',
               ...
               '2018-11-19 15:00:00', '2018-11-19 16:00:00',
               '2018-11-19 17:00:00', '2018-11-19 18:00:00',
               '2018-11-19 19:00:00', '2018-11-19 20:00:00',
               '2018-11-19 21:00:00', '2018-11-19 22:00:00',
               '2018-11-19 23:00:00', '2018-11-20 00:00:00'],
              dtype='datetime64[ns]', length=265, freq='H'))>),
        (rho_N,
         <function __main__.rho_N(t_N=DatetimeIndex(['2018-11-09 00:00:00', '2018-11-09 01:00:00',
               '2018-11-09 02:00:00', '2018-11-09 03:00:00',
               '2018-11-09 04:00:00', '2018-11-09 05:00:00',
          

In [7]:
fig = kamodo.plot('rho_N')

In this case, we only need to name the variable we wish to plot, because we have already defined a function $rho_N(t_N)$ with a default parameter for $t_N$. 

In [8]:
pio.write_image(fig, 'images/1d-time-series.svg')

![timeseries](images/1d-time-series.svg)

!!! note
    By providing default parameters, the function author can insure that anyone plotting the variable will not need to know where to place resolution!

## 2-D Parametric charts

For 2-D Plots, the output function must have input shape $(N,1)$ and output shape $(N,2)$.

In [9]:
from kamodo.kamodo import Kamodo
@kamodofy(units = 'cm')
def x_Ncomma2(theta_N = np.linspace(0,6*np.pi, 200)):
    r = theta_N
    x = r*np.cos(theta_N)
    y = r*np.sin(theta_N)
    return np.array(list(zip(x,y)))

kamodo = Kamodo(x_Ncomma2 = x_Ncomma2)
kamodo

Kamodo([(x_Ncomma2(theta_N),
         <function __main__.x_Ncomma2(theta_N=array([ 0.        ,  0.09472139,  0.18944277,  0.28416416,  0.37888555,
        0.47360693,  0.56832832,  0.66304971,  0.75777109,  0.85249248,
        0.94721387,  1.04193525,  1.13665664,  1.23137803,  1.32609941,
        1.4208208 ,  1.51554218,  1.61026357,  1.70498496,  1.79970634,
        1.89442773,  1.98914912,  2.0838705 ,  2.17859189,  2.27331328,
        2.36803466,  2.46275605,  2.55747744,  2.65219882,  2.74692021,
        2.8416416 ,  2.93636298,  3.03108437,  3.12580576,  3.22052714,
        3.31524853,  3.40996992,  3.5046913 ,  3.59941269,  3.69413408,
        3.78885546,  3.88357685,  3.97829823,  4.07301962,  4.16774101,
        4.26246239,  4.35718378,  4.45190517,  4.54662655,  4.64134794,
        4.73606933,  4.83079071,  4.9255121 ,  5.02023349,  5.11495487,
        5.20967626,  5.30439765,  5.39911903,  5.49384042,  5.58856181,
        5.68328319,  5.77800458,  5.87272597,  5.96744735,  6

Here, we again provide a default array for $\theta_N$ so the end user does not need to:

In [10]:
fig = kamodo.plot('x_Ncomma2')
pio.write_image(fig, 'images/fig-2d.svg')

![param2d](images/fig-2d.svg)

## 3-Dimensional parametric curves

For 3-D parametric curves, the output function must have input shape $(N,1)$ and output shape $(N,3)$.

In [11]:
@kamodofy(units = 'km')
def x_Ncomma3(t_N = pd.date_range('Nov 12, 2018', 'Dec 30, 2018', freq = '4 H')):
    dt_days = (t_N - t_N[0]).total_seconds()/(24*3600)
    theta = dt_days*np.pi/5
    r = theta
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    z = r
    return np.array(list(zip(x,y,z)))

kamodo = Kamodo(x_Ncomma3 = x_Ncomma3)
kamodo

Kamodo([(x_Ncomma3(t_N),
         <function __main__.x_Ncomma3(t_N=DatetimeIndex(['2018-11-12 00:00:00', '2018-11-12 04:00:00',
               '2018-11-12 08:00:00', '2018-11-12 12:00:00',
               '2018-11-12 16:00:00', '2018-11-12 20:00:00',
               '2018-11-13 00:00:00', '2018-11-13 04:00:00',
               '2018-11-13 08:00:00', '2018-11-13 12:00:00',
               ...
               '2018-12-28 12:00:00', '2018-12-28 16:00:00',
               '2018-12-28 20:00:00', '2018-12-29 00:00:00',
               '2018-12-29 04:00:00', '2018-12-29 08:00:00',
               '2018-12-29 12:00:00', '2018-12-29 16:00:00',
               '2018-12-29 20:00:00', '2018-12-30 00:00:00'],
              dtype='datetime64[ns]', length=289, freq='4H'))>),
        (x_Ncomma3,
         <function __main__.x_Ncomma3(t_N=DatetimeIndex(['2018-11-12 00:00:00', '2018-11-12 04:00:00',
               '2018-11-12 08:00:00', '2018-11-12 12:00:00',
               '2018-11-12 16:00:00', '2018-11-12 20:0

In [12]:
fig = kamodo.plot('x_Ncomma3')
pio.write_image(fig, 'images/3d-line.svg')

![3d-line](images/3d-line.svg)

Functions of three N-d arrays are also interpreted as 3D parametric plots, but with an additonal color component.

In [13]:
s = np.linspace(0, 8*np.pi, 100)
x = 10*np.sin(s/8)
y = 10*np.sin(s)
z = s

@kamodofy(units = 'kg')
def f_N(x_N = x, y_N = y, z_N = z):
    return x_N**2+y_N**2+z_N**2

kamodo = Kamodo(f_N = f_N)
kamodo

Kamodo([(f_N(x_N, y_N, z_N),
         <function __main__.f_N(x_N=array([0.00000000e+00, 3.17279335e-01, 6.34239197e-01, 9.50560433e-01,
       1.26592454e+00, 1.58001396e+00, 1.89251244e+00, 2.20310533e+00,
       2.51147987e+00, 2.81732557e+00, 3.12033446e+00, 3.42020143e+00,
       3.71662456e+00, 4.00930535e+00, 4.29794912e+00, 4.58226522e+00,
       4.86196736e+00, 5.13677392e+00, 5.40640817e+00, 5.67059864e+00,
       5.92907929e+00, 6.18158986e+00, 6.42787610e+00, 6.66769001e+00,
       6.90079011e+00, 7.12694171e+00, 7.34591709e+00, 7.55749574e+00,
       7.76146464e+00, 7.95761841e+00, 8.14575952e+00, 8.32569855e+00,
       8.49725430e+00, 8.66025404e+00, 8.81453363e+00, 8.95993774e+00,
       9.09631995e+00, 9.22354294e+00, 9.34147860e+00, 9.45000819e+00,
       9.54902241e+00, 9.63842159e+00, 9.71811568e+00, 9.78802446e+00,
       9.84807753e+00, 9.89821442e+00, 9.93838464e+00, 9.96854776e+00,
       9.98867339e+00, 9.99874128e+00, 9.99874128e+00, 9.98867339e+00,
       9.968

In [14]:
fig = kamodo.plot('f_N')
pio.write_image(fig, 'images/3d-points.svg')

![3d-points](images/3d-points.svg)

# Vector fields

Kamodo generates a 2-d vector (quiver) plot for functions of one variable, if both the input and output have shape (N,2). The input positions are assumed to be $x$,$y$ and the output vectors are assumed to be $v_x$, $v_y$

In [15]:
theta_N = np.linspace(0,6*np.pi, 200)
r = theta_N
x = r*np.cos(theta_N)
y = r*np.sin(theta_N)
points = np.array(list(zip(x,y)))

@kamodofy(units = 'cm')
def fvec_Ncomma2(rvec_Ncomma2 = points):
    return rvec_Ncomma2

kamodo = Kamodo(fvec_Ncomma2 = fvec_Ncomma2)
kamodo

Kamodo([(fvec_Ncomma2(rvec_Ncomma2),
         <function __main__.fvec_Ncomma2(rvec_Ncomma2=array([[ 0.00000000e+00,  0.00000000e+00],
       [ 9.42967773e-02,  8.95873053e-03],
       [ 1.86053513e-01,  3.56742843e-02],
       [ 2.72768131e-01,  7.96669081e-02],
       [ 3.52014010e-01,  1.40144188e-01],
       [ 4.21476526e-01,  2.16011724e-01],
       [ 4.78988182e-01,  3.05887888e-01],
       [ 5.22561895e-01,  4.08122504e-01],
       [ 5.50422000e-01,  5.20819211e-01],
       [ 5.61032587e-01,  6.41861249e-01],
       [ 5.53122803e-01,  7.68940356e-01],
       [ 5.25708787e-01,  8.99588429e-01],
       [ 4.78111944e-01,  1.03121156e+00],
       [ 4.09973321e-01,  1.16112606e+00],
       [ 3.21263867e-01,  1.28659597e+00],
       [ 2.12290428e-01,  1.40487171e+00],
       [ 8.36973798e-02,  1.51322928e+00],
       [-6.35361682e-02,  1.60900961e+00],
       [-2.28103595e-01,  1.68965750e+00],
       [-4.08382361e-01,  1.75275976e+00],
       [-6.02449873e-01,  1.79608201e+00],
      

In [16]:
fig = kamodo.plot('fvec_Ncomma2')
pio.write_image(fig, 'images/fig2d-vector.svg')

![fig2dvec](images/fig2d-vector.svg)

If we wish to represent a grid of vectors, we must first unravel the grid as a string of points.

In [17]:
x = np.linspace(-np.pi, np.pi, 25)
y = np.linspace(-np.pi, np.pi, 30)
xx, yy = np.meshgrid(x,y)
points = np.array(list(zip(xx.ravel(), yy.ravel())))

def fvec_Ncomma2(rvec_Ncomma2 = points):
    ux = np.sin(rvec_Ncomma2[:,0])
    uy = np.cos(rvec_Ncomma2[:,1])
    return np.vstack((ux,uy)).T
    
kamodo = Kamodo(fvec_Ncomma2 = fvec_Ncomma2)
kamodo

Kamodo([(fvec_Ncomma2(rvec_Ncomma2),
         <function __main__.fvec_Ncomma2(rvec_Ncomma2=array([[-3.14159265, -3.14159265],
       [-2.87979327, -3.14159265],
       [-2.61799388, -3.14159265],
       ...,
       [ 2.61799388,  3.14159265],
       [ 2.87979327,  3.14159265],
       [ 3.14159265,  3.14159265]]))>),
        (fvec_Ncomma2,
         <function __main__.fvec_Ncomma2(rvec_Ncomma2=array([[-3.14159265, -3.14159265],
       [-2.87979327, -3.14159265],
       [-2.61799388, -3.14159265],
       ...,
       [ 2.61799388,  3.14159265],
       [ 2.87979327,  3.14159265],
       [ 3.14159265,  3.14159265]]))>)])

In [18]:
fig = kamodo.plot('fvec_Ncomma2')
pio.write_image(fig, 'images/fig2d-vector-field.svg')

![fig2dvec](images/fig2d-vector-field.svg)

### 3D vector fields

Functions representing 3D vector fields should have one argument of shape (N,3) and an output shape of (N,3)

In [19]:
x, y, z = np.meshgrid(np.linspace(-2,2,4),
                      np.linspace(-3,3,6),
                      np.linspace(-5,5,10))
points = np.array(list(zip(x.ravel(), y.ravel(), z.ravel())))
def fvec_Ncomma3(rvec_Ncomma3 = points):
    return rvec_Ncomma3

kamodo = Kamodo(fvec_Ncomma3 = fvec_Ncomma3)
kamodo

Kamodo([(fvec_Ncomma3(rvec_Ncomma3),
         <function __main__.fvec_Ncomma3(rvec_Ncomma3=array([[-2.        , -3.        , -5.        ],
       [-2.        , -3.        , -3.88888889],
       [-2.        , -3.        , -2.77777778],
       [-2.        , -3.        , -1.66666667],
       [-2.        , -3.        , -0.55555556],
       [-2.        , -3.        ,  0.55555556],
       [-2.        , -3.        ,  1.66666667],
       [-2.        , -3.        ,  2.77777778],
       [-2.        , -3.        ,  3.88888889],
       [-2.        , -3.        ,  5.        ],
       [-0.66666667, -3.        , -5.        ],
       [-0.66666667, -3.        , -3.88888889],
       [-0.66666667, -3.        , -2.77777778],
       [-0.66666667, -3.        , -1.66666667],
       [-0.66666667, -3.        , -0.55555556],
       [-0.66666667, -3.        ,  0.55555556],
       [-0.66666667, -3.        ,  1.66666667],
       [-0.66666667, -3.        ,  2.77777778],
       [-0.66666667, -3.        ,  3.88888889

In [20]:
fig = kamodo.plot('fvec_Ncomma3')
pio.write_image(fig, 'images/fig3d-vector.svg')

![fig3dvec](images/fig3d-vector.svg)

# Contour plots

Scalar functions of two variables of size (N) and (M) and output size (N,M) will generate contour plots. Kamodo can handle both ```ij``` indexing and ```xy``` indexing.

In [21]:
from kamodo.kamodo import Kamodo
@kamodofy(units = 'cm^2')
def f_NcommaM(x_N = np.linspace(0, 8*np.pi,100), y_M = np.linspace(0, 5, 90)):
    x, y = np.meshgrid(x_N, y_M, indexing = 'xy')
    return np.sin(x)*y

kamodo = Kamodo(f_NcommaM = f_NcommaM)
kamodo

Kamodo([(f_NcommaM(x_N, y_M),
         <function __main__.f_NcommaM(x_N=array([ 0.        ,  0.25386607,  0.50773215,  0.76159822,  1.01546429,
        1.26933037,  1.52319644,  1.77706251,  2.03092858,  2.28479466,
        2.53866073,  2.7925268 ,  3.04639288,  3.30025895,  3.55412502,
        3.8079911 ,  4.06185717,  4.31572324,  4.56958931,  4.82345539,
        5.07732146,  5.33118753,  5.58505361,  5.83891968,  6.09278575,
        6.34665183,  6.6005179 ,  6.85438397,  7.10825004,  7.36211612,
        7.61598219,  7.86984826,  8.12371434,  8.37758041,  8.63144648,
        8.88531256,  9.13917863,  9.3930447 ,  9.64691077,  9.90077685,
       10.15464292, 10.40850899, 10.66237507, 10.91624114, 11.17010721,
       11.42397329, 11.67783936, 11.93170543, 12.1855715 , 12.43943758,
       12.69330365, 12.94716972, 13.2010358 , 13.45490187, 13.70876794,
       13.96263402, 14.21650009, 14.47036616, 14.72423224, 14.97809831,
       15.23196438, 15.48583045, 15.73969653, 15.9935626 , 16.24

In [22]:
fig = kamodo.plot('f_NcommaM')
pio.write_image(fig, 'images/fig2d-contour.svg')

![contour](images/fig2d-contour.svg)

Since $x_N$ and $y_M$ have differnt sizes, we could have used ```indexing=ij``` as an argument to meshgrid and kamodo would have produced the same figure - Kamodo swaps the ordering where appropriate. In the event that *both* arguments have the same size, we can pass an ```indexing``` argument as an option to the plot function.

In [23]:
@kamodofy(units = 'cm^2')
def f_NN(x_N = np.linspace(0, 8*np.pi, 90), y_N = np.linspace(0, 5, 90)):
    x, y = np.meshgrid(x_N, y_N, indexing = 'xy')
    return np.sin(x)*y

kamodo = Kamodo(f_NN = f_NN)
kamodo

Kamodo([(f_NN(x_N, y_N),
         <function __main__.f_NN(x_N=array([ 0.        ,  0.28239035,  0.5647807 ,  0.84717105,  1.1295614 ,
        1.41195175,  1.69434211,  1.97673246,  2.25912281,  2.54151316,
        2.82390351,  3.10629386,  3.38868421,  3.67107456,  3.95346491,
        4.23585526,  4.51824561,  4.80063597,  5.08302632,  5.36541667,
        5.64780702,  5.93019737,  6.21258772,  6.49497807,  6.77736842,
        7.05975877,  7.34214912,  7.62453947,  7.90692982,  8.18932018,
        8.47171053,  8.75410088,  9.03649123,  9.31888158,  9.60127193,
        9.88366228, 10.16605263, 10.44844298, 10.73083333, 11.01322368,
       11.29561404, 11.57800439, 11.86039474, 12.14278509, 12.42517544,
       12.70756579, 12.98995614, 13.27234649, 13.55473684, 13.83712719,
       14.11951754, 14.4019079 , 14.68429825, 14.9666886 , 15.24907895,
       15.5314693 , 15.81385965, 16.09625   , 16.37864035, 16.6610307 ,
       16.94342105, 17.2258114 , 17.50820175, 17.79059211, 18.07298246,
  

In [24]:
fig = kamodo.plot(f_NN = dict(indexing = 'xy'))
pio.write_image(fig, 'images/fig2d-contour-xy.svg')

![fig2dcontourxy](images/fig2d-contour-xy.svg)

# Skew (Carpet) Plots

Functions of two arguments each having shape (N,M) matching the output shape will produce skewed contour plots, whereby the x and y components of the grid are independent.

In [25]:
r = np.linspace(1, 3, 20)
theta = np.linspace(0, np.pi, 14)
r_, theta_ = np.meshgrid(r,theta)
XX = r_*np.cos(theta_)
YY = r_*np.sin(theta_)

@kamodofy(units = 'cm^2')
def f_NM(x_NM = XX, y_NM = YY):
    return np.sin(x_NM)+y_NM

kamodo = Kamodo(f_NM = f_NM)
kamodo

Kamodo([(f_NM(x_NM, y_NM),
         <function __main__.f_NM(x_NM=array([[ 1.        ,  1.10526316,  1.21052632,  1.31578947,  1.42105263,
         1.52631579,  1.63157895,  1.73684211,  1.84210526,  1.94736842,
         2.05263158,  2.15789474,  2.26315789,  2.36842105,  2.47368421,
         2.57894737,  2.68421053,  2.78947368,  2.89473684,  3.        ],
       [ 0.97094182,  1.07314622,  1.17535062,  1.27755502,  1.37975942,
         1.48196383,  1.58416823,  1.68637263,  1.78857703,  1.89078143,
         1.99298584,  2.09519024,  2.19739464,  2.29959904,  2.40180344,
         2.50400784,  2.60621225,  2.70841665,  2.81062105,  2.91282545],
       [ 0.88545603,  0.97866192,  1.07186782,  1.16507372,  1.25827962,
         1.35148551,  1.44469141,  1.53789731,  1.63110321,  1.7243091 ,
         1.817515  ,  1.9107209 ,  2.00392679,  2.09713269,  2.19033859,
         2.28354449,  2.37675038,  2.46995628,  2.56316218,  2.65636808],
       [ 0.74851075,  0.82730135,  0.90609196,  0.984882

In [26]:
fig = kamodo.plot('f_NM')
pio.write_image(fig, 'images/fig2d-skew.svg')

![fig2dskew](images/fig2d-skew.svg)

# Parametric surfaces

To generate a purely geometrical parametric surface, supply a functions of three variables, each of size (N,M) and of output shape (1).

In [27]:
from kamodo.kamodo import Kamodo
u = np.linspace(-2, 2, 40)
v = np.linspace(-2, 2, 50)
uu, vv = np.meshgrid(u,v)

@kamodofy(units = 'cm')
def parametric(x_NM = uu*np.sin(vv*np.pi), 
               y_NM = vv, 
               z_NM = np.exp(-uu**2-vv**2)):
    return np.array([1])
kamodo = Kamodo(p = parametric)
kamodo

Kamodo([(p(x_NM, y_NM, z_NM),
         <function __main__.parametric(x_NM=array([[-4.89858720e-16, -4.64737760e-16, -4.39616800e-16, ...,
         4.39616800e-16,  4.64737760e-16,  4.89858720e-16],
       [-5.07309168e-01, -4.81293313e-01, -4.55277458e-01, ...,
         4.55277458e-01,  4.81293313e-01,  5.07309168e-01],
       [-9.81435104e-01, -9.31105099e-01, -8.80775093e-01, ...,
         8.80775093e-01,  9.31105099e-01,  9.81435104e-01],
       ...,
       [ 9.81435104e-01,  9.31105099e-01,  8.80775093e-01, ...,
        -8.80775093e-01, -9.31105099e-01, -9.81435104e-01],
       [ 5.07309168e-01,  4.81293313e-01,  4.55277458e-01, ...,
        -4.55277458e-01, -4.81293313e-01, -5.07309168e-01],
       [ 4.89858720e-16,  4.64737760e-16,  4.39616800e-16, ...,
        -4.39616800e-16, -4.64737760e-16, -4.89858720e-16]]), y_NM=array([[-2.        , -2.        , -2.        , ..., -2.        ,
        -2.        , -2.        ],
       [-1.91836735, -1.91836735, -1.91836735, ..., -1.91836735

In [28]:
fig = kamodo.plot('p')
pio.write_image(fig, 'images/3d-parametric.svg')

![3dparametric](images/3d-parametric.svg)

To control the color of the parametric surface, have the output shape be (N,M).

In [29]:
R = 1
theta = np.linspace(.2*np.pi, .8*np.pi, 40)
phi = np.linspace(0, 2*np.pi, 50)
theta_, phi_ = np.meshgrid(theta, phi)
r = (R +.1*(np.cos(10*theta_)*np.sin(14*phi_)))

xx = r*np.sin(theta_)*np.cos(phi_)
yy = r*np.sin(theta_)*np.sin(phi_)
zz = r*np.cos(theta_)

@kamodofy(units = 'cm')
def spherelike(x_NM = xx, y_NM = yy, z_NM = zz):
    return .1*x_NM + x_NM**2 + y_NM**2 + z_NM**2

kamodo = Kamodo(h_NM = spherelike)
kamodo

Kamodo([(h_NM(x_NM, y_NM, z_NM),
         <function __main__.spherelike(x_NM=array([[0.58778525, 0.6261852 , 0.66312266, ..., 0.66312266, 0.6261852 ,
        0.58778525],
       [0.6397939 , 0.67465623, 0.69410212, ..., 0.69410212, 0.67465623,
        0.6397939 ],
       [0.54389269, 0.58243542, 0.62562543, ..., 0.62562543, 0.58243542,
        0.54389269],
       ...,
       [0.59323062, 0.62897603, 0.65724485, ..., 0.65724485, 0.62897603,
        0.59323062],
       [0.52612519, 0.56743222, 0.62125474, ..., 0.62125474, 0.56743222,
        0.52612519],
       [0.58778525, 0.6261852 , 0.66312266, ..., 0.66312266, 0.6261852 ,
        0.58778525]]), y_NM=array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.24922881e-02,  8.69872880e-02,  8.94945582e-02, ...,
         8.94945582e-02,  8.69872880e-02,  8.24922881e-02],
       [ 1.42625459e-01,  1.52732554e-01,  1.64058307e-01, ...,
         1.64058307e-01,  1.

In [30]:
fig = kamodo.plot('h_NM')
pio.write_image(fig, 'images/3d-parametric-color.svg')

![parametric3dcolor](images/3d-parametric-color.svg)

## Map-to-plane
We often need to produce slices through a volumetric grid of data. This may be accomplished through the use of volumetric grid interpolators equipped with default values for each of the input arguments. Suppose such a function has default input arguments of size (L), (M), (N), and output shape (L,M,N), then a cartesian plane will be generated if the user overrides one of these defaults (e.g. setting $L = 1$).

In [31]:
@kamodofy(units = 'g/cm^3')
def f_LMN(
      x_L = np.linspace(-5, 5, 50), 
      y_M = np.linspace(0, 10, 75), 
      z_N = np.linspace(-20, 20, 100)):
    xx, yy, zz = np.meshgrid(x_L,y_M,z_N, indexing = 'xy')
    return xx + yy + zz

kamodo = Kamodo(f_LMN = f_LMN)
kamodo

Kamodo([(f_LMN(x_L, y_M, z_N),
         <function __main__.f_LMN(x_L=array([-5.        , -4.79591837, -4.59183673, -4.3877551 , -4.18367347,
       -3.97959184, -3.7755102 , -3.57142857, -3.36734694, -3.16326531,
       -2.95918367, -2.75510204, -2.55102041, -2.34693878, -2.14285714,
       -1.93877551, -1.73469388, -1.53061224, -1.32653061, -1.12244898,
       -0.91836735, -0.71428571, -0.51020408, -0.30612245, -0.10204082,
        0.10204082,  0.30612245,  0.51020408,  0.71428571,  0.91836735,
        1.12244898,  1.32653061,  1.53061224,  1.73469388,  1.93877551,
        2.14285714,  2.34693878,  2.55102041,  2.75510204,  2.95918367,
        3.16326531,  3.36734694,  3.57142857,  3.7755102 ,  3.97959184,
        4.18367347,  4.3877551 ,  4.59183673,  4.79591837,  5.        ]), y_M=array([ 0.        ,  0.13513514,  0.27027027,  0.40540541,  0.54054054,
        0.67567568,  0.81081081,  0.94594595,  1.08108108,  1.21621622,
        1.35135135,  1.48648649,  1.62162162,  1.75675676,  1

In [32]:
fig = kamodo.plot(f_LMN = dict(z_N = -5))
pio.write_image(fig,'images/fig2d-map-to-plane.svg')

![maptoplane](images/fig2d-map-to-plane.svg)

!!!tip
    By providing appropriate defaults for the undelying grid structure, the interpolator author can ensure that the user can generate figures with optimal resolution!

# Multiple traces

Kamodo supports multiple traces in the same figure. Simply provide ```plot``` with multiple function-argument pairs.

In [33]:
from kamodo.kamodo import Kamodo
t_N = pd.date_range('Nov 9, 2018', 'Nov 20, 2018', freq = 'H')

@kamodofy(units = 'kg/m^3')
def rho_N(t_N = t_N):
    dt_days = (t_N - t_N[0]).total_seconds()/(24*3600)
    return 1+np.sin(dt_days) + .1*np.random.random(len(dt_days))

@kamodofy(units = 'nPa')
def p_N(t_N = t_N):
    dt_days = (t_N - t_N[0]).total_seconds()/(24*3600)
    return 1+np.sin(2*dt_days) + .1*np.random.random(len(dt_days))


kamodo = Kamodo(rho_N = rho_N, p_N = p_N, verbose = False)
kamodo

Kamodo([(rho_N(t_N),
         <function __main__.rho_N(t_N=DatetimeIndex(['2018-11-09 00:00:00', '2018-11-09 01:00:00',
               '2018-11-09 02:00:00', '2018-11-09 03:00:00',
               '2018-11-09 04:00:00', '2018-11-09 05:00:00',
               '2018-11-09 06:00:00', '2018-11-09 07:00:00',
               '2018-11-09 08:00:00', '2018-11-09 09:00:00',
               ...
               '2018-11-19 15:00:00', '2018-11-19 16:00:00',
               '2018-11-19 17:00:00', '2018-11-19 18:00:00',
               '2018-11-19 19:00:00', '2018-11-19 20:00:00',
               '2018-11-19 21:00:00', '2018-11-19 22:00:00',
               '2018-11-19 23:00:00', '2018-11-20 00:00:00'],
              dtype='datetime64[ns]', length=265, freq='H'))>),
        (rho_N,
         <function __main__.rho_N(t_N=DatetimeIndex(['2018-11-09 00:00:00', '2018-11-09 01:00:00',
               '2018-11-09 02:00:00', '2018-11-09 03:00:00',
               '2018-11-09 04:00:00', '2018-11-09 05:00:00',
          

In [34]:
fig = kamodo.plot('p_N','rho_N')
pio.write_image(fig, 'images/multi-trace.svg')

![multitrace](images/multi-trace.svg)

!!! note
    Plot types must be compatible for kamodo to plot different variables on the same axes.

Kamodo can also handle multiple traces in 3D

In [35]:
from kamodo.kamodo import Kamodo, kamodofy

@kamodofy(units = 'g/cm^3')
def f_LMN(
      x_L = np.linspace(-5, 5, 50), 
      y_M = np.linspace(0, 10, 75), 
      z_N = np.linspace(-20, 20, 100)):
    xx, yy, zz = np.meshgrid(x_L,y_M,z_N, indexing = 'xy')
    return xx + yy + zz

kamodo = Kamodo(f_LMN = f_LMN, g_LMN = f_LMN)
kamodo

Kamodo([(f_LMN(x_L, y_M, z_N),
         <function __main__.f_LMN(x_L=array([-5.        , -4.79591837, -4.59183673, -4.3877551 , -4.18367347,
       -3.97959184, -3.7755102 , -3.57142857, -3.36734694, -3.16326531,
       -2.95918367, -2.75510204, -2.55102041, -2.34693878, -2.14285714,
       -1.93877551, -1.73469388, -1.53061224, -1.32653061, -1.12244898,
       -0.91836735, -0.71428571, -0.51020408, -0.30612245, -0.10204082,
        0.10204082,  0.30612245,  0.51020408,  0.71428571,  0.91836735,
        1.12244898,  1.32653061,  1.53061224,  1.73469388,  1.93877551,
        2.14285714,  2.34693878,  2.55102041,  2.75510204,  2.95918367,
        3.16326531,  3.36734694,  3.57142857,  3.7755102 ,  3.97959184,
        4.18367347,  4.3877551 ,  4.59183673,  4.79591837,  5.        ]), y_M=array([ 0.        ,  0.13513514,  0.27027027,  0.40540541,  0.54054054,
        0.67567568,  0.81081081,  0.94594595,  1.08108108,  1.21621622,
        1.35135135,  1.48648649,  1.62162162,  1.75675676,  1

In [36]:
fig = kamodo.plot(f_LMN = dict(z_N = 0), 
                  g_LMN = dict(y_M = 5))

In [37]:
pio.write_image(fig, 'images/multi-trace3d.svg')

![multi3d](images/multi-trace3d.svg?1)

!!! bug
    Multiple traces results in different colorbars which may overlap. More control over the layout will be available in future updates. 